## Basic Composable Stable Testing

In [1]:
from stableswappy.cst.factory import StableswapFactory
from stableswappy.cst.exchg import StableswapPoolMath
from stableswappy.group import StableswapERC20Group
from stableswappy.erc import StableswapERC20

### Parameters

In [2]:
USER = 'user_test'

AMPL_COEFF = 2000 

amt_dai = 79566307.559825807715868071
decimal_dai = 18

amt_usdc = 81345068.187939
decimal_usdc = 6

amt_usdt = 55663250.772939
decimal_usdt = 6

### Tokens

In [3]:
dai = StableswapERC20("DAI", "0xA0b")
dai.set_params(amt_dai, decimal_dai)

usdc = StableswapERC20("USDC", "0xf93")
usdc.set_params(amt_usdc, decimal_usdc)

usdt = StableswapERC20("USDT", "0xd7c")
usdt.set_params(amt_usdt, decimal_usdt)

sgrp = StableswapERC20Group()
sgrp.add_token(dai)
sgrp.add_token(usdc)
sgrp.add_token(usdt)

### Join stableswap pool
* Join all assets

In [4]:
sfactory = StableswapFactory("Pool factory", "0x2")
lp = sfactory.create_exchange(sgrp, symbol="LP", address="0x011")
lp.join_pool(sgrp, AMPL_COEFF, USER)
lp.info()

Stableswap Exchange: DAI-USDC-USDT (LP)
Coins: DAI/USDC/USDT
Reserves: DAI = 79566307.56 | USDC = 81345068.19 | USDT = 55663250.77
Liquidity: 216573027.91811988 



### Swap: USDC -> USDT

In [5]:
usdc_before = lp.get_reserve(usdc)
usdt_before = lp.get_reserve(usdt)

amt_tkn_in = 10000
tkn_in = usdc
tkn_out = usdt
res = lp.swap(amt_tkn_in, tkn_in, tkn_out, USER)

print(f"{amt_tkn_in} {tkn_in.token_name} was swapped for {res['tkn_out_amt']} {tkn_out.token_name}")

10000 USDC was swapped for 9996.862748 USDT


In [6]:
lp.info()

## Check for leaks
usdc_check = lp.get_reserve(usdc)-amt_tkn_in
usdt_check = lp.get_reserve(usdt)+res['tkn_out_amt']

usdc_test = 'PASS' if {usdc_before == usdc_check} else 'FAIL'
usdt_test = 'PASS' if {usdt_before == usdt_check} else 'FAIL'
print(f'Liquidity Leak (USDC): {usdc_test}')
print(f'Liquidity Leak (USDT): {usdt_test}')

Stableswap Exchange: DAI-USDC-USDT (LP)
Coins: DAI/USDC/USDT
Reserves: DAI = 79566307.56 | USDC = 81355068.19 | USDT = 55653253.91
Liquidity: 216573027.91811988 

Liquidity Leak (USDC): PASS
Liquidity Leak (USDT): PASS


### Swap: USDC -> DAI

In [7]:
usdc_before = lp.get_reserve(usdc)
dai_before = lp.get_reserve(dai)

amt_tkn_in = 10000
tkn_in = usdc
tkn_out = dai
res = lp.swap(amt_tkn_in, tkn_in, tkn_out, USER)

print(f"{amt_tkn_in} {tkn_in.token_name} was swapped for {res['tkn_out_amt']} {tkn_out.token_name}")

10000 USDC was swapped for 9998.895308918858 DAI


In [8]:
lp.info()

## Check for leaks
usdc_check = lp.get_reserve(usdc)-amt_tkn_in
dai_check = lp.get_reserve(dai)+res['tkn_out_amt']

usdc_test = 'PASS' if {usdc_before == usdc_check} else 'FAIL'
dai_test = 'PASS' if {dai_before == dai_check} else 'FAIL'
print(f'Liquidity Leak (USDC): {usdc_test}')
print(f'Liquidity Leak (DAI): {dai_test}')

Stableswap Exchange: DAI-USDC-USDT (LP)
Coins: DAI/USDC/USDT
Reserves: DAI = 79556308.66 | USDC = 81365068.19 | USDT = 55653253.91
Liquidity: 216573027.91811988 

Liquidity Leak (USDC): PASS
Liquidity Leak (DAI): PASS


In [9]:
lp.providers

{'user_test': 216573027.91811988}

In [10]:
math_pool = lp.get_pool()
math_pool.balances

[79556308664516891142540371, 81365068187939, 55653253910191]

In [11]:
sgrp.get_decimal_amts()

{'DAI': 79556308664516900000000000,
 'USDC': 81365068187939,
 'USDT': 55653253910191}

In [12]:
sgrp.get_rates()

{'DAI': 1000000000000000000,
 'USDC': 1000000000000000000000000000000,
 'USDT': 1000000000000000000000000000000}